# 🚇 Miami-Dade MPO Transit Accessibility - EDA

**Deloitte Capstone Project: AI for Equitable Transport**

This notebook performs Exploratory Data Analysis on Miami-Dade transportation accessibility data.

## Dataset Overview
- **6 GeoPackage files** covering different transportation modes
- **36,507 census blocks** in Miami-Dade County
- **Accessibility metrics** at 5-minute intervals (5 to 60 minutes)
- **LEHD/LODES employment data** integrated with accessibility

## Data Dictionary (LODES Columns)
| Prefix | Meaning |
|--------|--------|
| `w_` | Workplace (jobs located in block) |
| `r_` | Residence (workers living in block) |
| `c000` | Total jobs/workers |
| `ca01/02/03` | Age groups (≤29, 30-54, 55+) |
| `ce01/02/03` | Earnings ($1250/mo or less, $1251-3333, $3333+) |
| `cns01-20` | Industry sectors (NAICS) |
| `cr01-07` | Race categories |
| `cd01-04` | Education levels |
| `_18/_19` | Year (2018, 2019) |

---
## 1. Setup & Configuration

In [ ]:
# Install required packages (uncomment if running on Colab)
# !pip install pandas numpy matplotlib seaborn plotly -q

In [ ]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Set plot style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')

print("✅ Libraries loaded successfully!")

✅ Libraries loaded successfully!


In [ ]:
# ⚠️ UPDATE THIS PATH to where you uploaded your .gpkg files
# For Colab: upload files to Google Drive and mount, or upload directly

# Option 1: If files are in same directory as notebook
BASE_PATH = Path('.')

# Option 2: If using Google Drive (uncomment and modify)
# from google.colab import drive
# drive.mount('/content/drive')
# BASE_PATH = Path('/content/drive/MyDrive/YOUR_FOLDER_NAME/')

# File mapping
GPKG_FILES = {
    'automobile': '12197701_au_2021_08.gpkg',
    'bike_lts1': '12197701_bi_2021_1200_lts1.gpkg',
    'bike_lts2': '12197701_bi_2021_1200_lts2.gpkg',
    'bike_lts3': '12197701_bi_2021_1200_lts3.gpkg',
    'bike_lts4': '12197701_bi_2021_1200_lts4.gpkg',
    'transit': '12197701_tr_2021_0700-0859-avg.gpkg',
}

# Verify files exist
print("📁 Checking files...")
for name, file in GPKG_FILES.items():
    filepath = BASE_PATH / file
    if filepath.exists():
        size_mb = filepath.stat().st_size / (1024 * 1024)
        print(f"  ✅ {name:15}: {file} ({size_mb:.1f} MB)")
    else:
        print(f"  ❌ {name:15}: {file} NOT FOUND")

📁 Checking files...
  ❌ automobile     : 12197701_au_2021_08.gpkg NOT FOUND
  ❌ bike_lts1      : 12197701_bi_2021_1200_lts1.gpkg NOT FOUND
  ❌ bike_lts2      : 12197701_bi_2021_1200_lts2.gpkg NOT FOUND
  ❌ bike_lts3      : 12197701_bi_2021_1200_lts3.gpkg NOT FOUND
  ❌ bike_lts4      : 12197701_bi_2021_1200_lts4.gpkg NOT FOUND
  ❌ transit        : 12197701_tr_2021_0700-0859-avg.gpkg NOT FOUND


---
## 2. Helper Functions

In [ ]:
def load_accessibility_data(mode: str, time_minutes: int = 30) -> pd.DataFrame:
    """Load accessibility data for a specific mode and travel time."""
    filepath = BASE_PATH / GPKG_FILES[mode]
    conn = sqlite3.connect(filepath)

    # Table naming patterns
    if mode == 'automobile':
        table = f'au_08_{time_minutes}_minutes'
    elif mode.startswith('bike'):
        table = f'bi_{time_minutes}_minutes'
    else:  # transit
        table = f'tr_{time_minutes}_minutes'

    df = pd.read_sql(f"SELECT * FROM {table}", conn)
    conn.close()
    return df


def load_blocks(mode: str = 'automobile') -> pd.DataFrame:
    """Load census block data."""
    filepath = BASE_PATH / GPKG_FILES[mode]
    conn = sqlite3.connect(filepath)
    df = pd.read_sql("SELECT fid, blockid FROM blocks", conn)
    conn.close()
    return df


def compute_gini(values: np.ndarray) -> float:
    """Compute Gini coefficient for measuring inequality."""
    values = np.sort(values[~np.isnan(values)])
    n = len(values)
    if n == 0 or np.sum(values) == 0:
        return np.nan
    index = np.arange(1, n + 1)
    return (2 * np.sum(index * values) - (n + 1) * np.sum(values)) / (n * np.sum(values))


print("✅ Helper functions defined!")

✅ Helper functions defined!


---
## 3. Data Overview & Structure

In [ ]:
# Explore table structure in one of the files
conn = sqlite3.connect(BASE_PATH / GPKG_FILES['automobile'])
cursor = conn.cursor()

# List all tables
cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'gpkg_%' AND name NOT LIKE 'rtree_%' AND name != 'sqlite_sequence'")
tables = [row[0] for row in cursor.fetchall()]

print("📋 Tables in automobile gpkg:")
for t in tables:
    print(f"  - {t}")

conn.close()

📋 Tables in automobile gpkg:


In [ ]:
# Load sample data
transit_30 = load_accessibility_data('transit', 30)
print(f"📊 Transit 30-min data shape: {transit_30.shape}")
print(f"\n📋 Columns ({len(transit_30.columns)} total):")
print(transit_30.columns.tolist()[:20], "...")

DatabaseError: Execution failed on sql 'SELECT * FROM tr_30_minutes': no such table: tr_30_minutes

In [ ]:
# Preview key columns
key_cols = ['id', 'w_c000_19', 'w_ce01_19', 'w_ce02_19', 'w_ce03_19', 'r_c000_19']
transit_30[key_cols].head(10)

---
## 4. Mode Comparison Analysis

In [ ]:
# Compare all modes at 30-minute travel time
mode_comparison = []

for mode in GPKG_FILES.keys():
    df = load_accessibility_data(mode, 30)
    mode_comparison.append({
        'Mode': mode,
        'Avg Jobs': df['w_c000_19'].mean(),
        'Median Jobs': df['w_c000_19'].median(),
        'Max Jobs': df['w_c000_19'].max(),
        'Min Jobs': df['w_c000_19'].min(),
        'Std Dev': df['w_c000_19'].std(),
        'Zero Access Blocks': (df['w_c000_19'] == 0).sum(),
    })

comparison_df = pd.DataFrame(mode_comparison)
comparison_df

In [ ]:
# Visualize mode comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Average jobs by mode
ax1 = axes[0]
colors = ['#e74c3c', '#3498db', '#3498db', '#3498db', '#3498db', '#2ecc71']
bars = ax1.bar(comparison_df['Mode'], comparison_df['Avg Jobs'], color=colors)
ax1.set_ylabel('Average Jobs Accessible')
ax1.set_title('Average Jobs Accessible in 30 Minutes by Mode')
ax1.tick_params(axis='x', rotation=45)
ax1.set_yscale('log')

# Add value labels
for bar, val in zip(bars, comparison_df['Avg Jobs']):
    ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height(), f'{val:,.0f}',
             ha='center', va='bottom', fontsize=9)

# Zero access blocks
ax2 = axes[1]
ax2.bar(comparison_df['Mode'], comparison_df['Zero Access Blocks'], color=colors)
ax2.set_ylabel('Number of Blocks')
ax2.set_title('Census Blocks with Zero Job Access (30 min)')
ax2.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

---
## 5. Accessibility Curves by Travel Time

In [ ]:
def get_accessibility_curve(mode: str) -> pd.DataFrame:
    """Get accessibility at all time intervals for a mode."""
    times = [5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60]
    results = []

    for t in times:
        df = load_accessibility_data(mode, t)
        results.append({
            'time': t,
            'mean': df['w_c000_19'].mean(),
            'median': df['w_c000_19'].median(),
            'p10': df['w_c000_19'].quantile(0.10),
            'p90': df['w_c000_19'].quantile(0.90),
        })

    return pd.DataFrame(results)

# Generate curves
print("⏳ Generating accessibility curves (this may take a minute)...")
auto_curve = get_accessibility_curve('automobile')
transit_curve = get_accessibility_curve('transit')
bike_lts1_curve = get_accessibility_curve('bike_lts1')
bike_lts4_curve = get_accessibility_curve('bike_lts4')
print("✅ Done!")

In [ ]:
# Plot accessibility curves
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Auto vs Transit
ax1 = axes[0]
ax1.plot(auto_curve['time'], auto_curve['mean'], 'o-', label='Automobile', color='#e74c3c', linewidth=2)
ax1.plot(transit_curve['time'], transit_curve['mean'], 's-', label='Transit', color='#2ecc71', linewidth=2)
ax1.fill_between(auto_curve['time'], auto_curve['p10'], auto_curve['p90'], alpha=0.2, color='#e74c3c')
ax1.fill_between(transit_curve['time'], transit_curve['p10'], transit_curve['p90'], alpha=0.2, color='#2ecc71')
ax1.set_xlabel('Travel Time (minutes)')
ax1.set_ylabel('Jobs Accessible')
ax1.set_title('Auto vs Transit Accessibility')
ax1.legend()
ax1.set_yscale('log')
ax1.grid(True, alpha=0.3)

# Bike LTS comparison
ax2 = axes[1]
ax2.plot(bike_lts1_curve['time'], bike_lts1_curve['mean'], 'o-', label='LTS1 (Safe)', color='#27ae60', linewidth=2)
ax2.plot(bike_lts4_curve['time'], bike_lts4_curve['mean'], 's-', label='LTS4 (High Stress)', color='#e67e22', linewidth=2)
ax2.fill_between(bike_lts1_curve['time'], bike_lts1_curve['p10'], bike_lts1_curve['p90'], alpha=0.2, color='#27ae60')
ax2.fill_between(bike_lts4_curve['time'], bike_lts4_curve['p10'], bike_lts4_curve['p90'], alpha=0.2, color='#e67e22')
ax2.set_xlabel('Travel Time (minutes)')
ax2.set_ylabel('Jobs Accessible')
ax2.set_title('Bicycle Accessibility: Safe vs High-Stress Routes')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Calculate and display the auto-transit gap
gap_analysis = pd.DataFrame({
    'Travel Time': auto_curve['time'],
    'Auto (mean jobs)': auto_curve['mean'],
    'Transit (mean jobs)': transit_curve['mean'],
})
gap_analysis['Gap Ratio'] = gap_analysis['Auto (mean jobs)'] / gap_analysis['Transit (mean jobs)']

print("🚗🚇 AUTO vs TRANSIT GAP ANALYSIS")
print("=" * 60)
gap_analysis.round(0)

---
## 6. Transit Equity Deep Dive

In [ ]:
# Load transit data for equity analysis
transit_30 = load_accessibility_data('transit', 30)

# Basic statistics
print("📊 TRANSIT ACCESSIBILITY STATISTICS (30 min)")
print("=" * 50)
print(f"Total census blocks: {len(transit_30):,}")
print(f"Mean jobs accessible: {transit_30['w_c000_19'].mean():,.0f}")
print(f"Median jobs accessible: {transit_30['w_c000_19'].median():,.0f}")
print(f"Std deviation: {transit_30['w_c000_19'].std():,.0f}")
print(f"Blocks with zero access: {(transit_30['w_c000_19'] == 0).sum():,}")

# Gini coefficient
gini = compute_gini(transit_30['w_c000_19'].values)
print(f"\n📈 Gini Coefficient: {gini:.3f}")
print("   (0 = perfect equality, 1 = perfect inequality)")

In [ ]:
# Distribution analysis
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram
ax1 = axes[0]
ax1.hist(transit_30['w_c000_19'], bins=50, color='#2ecc71', edgecolor='white', alpha=0.7)
ax1.axvline(transit_30['w_c000_19'].mean(), color='red', linestyle='--', label=f"Mean: {transit_30['w_c000_19'].mean():,.0f}")
ax1.axvline(transit_30['w_c000_19'].median(), color='blue', linestyle='--', label=f"Median: {transit_30['w_c000_19'].median():,.0f}")
ax1.set_xlabel('Jobs Accessible (30 min transit)')
ax1.set_ylabel('Number of Blocks')
ax1.set_title('Distribution of Transit Job Accessibility')
ax1.legend()

# Lorenz curve for inequality
ax2 = axes[1]
sorted_jobs = np.sort(transit_30['w_c000_19'].values)
cumulative_jobs = np.cumsum(sorted_jobs) / np.sum(sorted_jobs)
cumulative_pop = np.arange(1, len(sorted_jobs) + 1) / len(sorted_jobs)

ax2.plot(cumulative_pop, cumulative_jobs, color='#2ecc71', linewidth=2, label='Transit Access')
ax2.plot([0, 1], [0, 1], 'k--', label='Perfect Equality')
ax2.fill_between(cumulative_pop, cumulative_jobs, cumulative_pop, alpha=0.3, color='#e74c3c')
ax2.set_xlabel('Cumulative Share of Blocks')
ax2.set_ylabel('Cumulative Share of Job Access')
ax2.set_title(f'Lorenz Curve (Gini = {gini:.3f})')
ax2.legend()
ax2.set_xlim(0, 1)
ax2.set_ylim(0, 1)

plt.tight_layout()
plt.show()

In [ ]:
# Percentile analysis
percentiles = [5, 10, 25, 50, 75, 90, 95, 99]
percentile_df = pd.DataFrame({
    'Percentile': [f'P{p}' for p in percentiles],
    'Jobs Accessible': [transit_30['w_c000_19'].quantile(p/100) for p in percentiles]
})

print("📈 TRANSIT ACCESSIBILITY BY PERCENTILE (30 min)")
print("=" * 40)
percentile_df['Jobs Accessible'] = percentile_df['Jobs Accessible'].apply(lambda x: f"{x:,.0f}")
print(percentile_df.to_string(index=False))

# Highlight disparity
p10 = transit_30['w_c000_19'].quantile(0.10)
p90 = transit_30['w_c000_19'].quantile(0.90)
print(f"\n⚠️ P90/P10 Ratio: {p90/p10:.1f}x")
print("   (Top decile accesses this many times more jobs than bottom decile)")

---
## 7. Wage-Based Accessibility Analysis

In [ ]:
# Compare accessibility to different wage levels
wage_analysis = pd.DataFrame({
    'Wage Category': ['Low (≤$1,250/mo)', 'Medium ($1,251-3,333/mo)', 'High (≥$3,333/mo)'],
    'Transit (30 min)': [
        transit_30['w_ce01_19'].mean(),
        transit_30['w_ce02_19'].mean(),
        transit_30['w_ce03_19'].mean()
    ]
})

# Add auto for comparison
auto_30 = load_accessibility_data('automobile', 30)
wage_analysis['Auto (30 min)'] = [
    auto_30['w_ce01_19'].mean(),
    auto_30['w_ce02_19'].mean(),
    auto_30['w_ce03_19'].mean()
]

print("💰 WAGE-BASED JOB ACCESSIBILITY")
print("=" * 50)
wage_analysis

In [ ]:
# Visualize wage accessibility
fig, ax = plt.subplots(figsize=(10, 6))

x = np.arange(len(wage_analysis))
width = 0.35

bars1 = ax.bar(x - width/2, wage_analysis['Transit (30 min)'], width, label='Transit', color='#2ecc71')
bars2 = ax.bar(x + width/2, wage_analysis['Auto (30 min)'], width, label='Automobile', color='#e74c3c')

ax.set_ylabel('Average Jobs Accessible')
ax.set_title('Job Accessibility by Wage Level (30 min travel)')
ax.set_xticks(x)
ax.set_xticklabels(wage_analysis['Wage Category'])
ax.legend()

# Add value labels
for bar in bars1:
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height(), f'{bar.get_height():,.0f}',
            ha='center', va='bottom', fontsize=9)
for bar in bars2:
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height(), f'{bar.get_height():,.0f}',
            ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()

---
## 8. Zero-Access Analysis

In [ ]:
# Analyze zero-access blocks across travel times
zero_access = []

for time in [15, 30, 45, 60]:
    transit_df = load_accessibility_data('transit', time)
    auto_df = load_accessibility_data('automobile', time)

    zero_access.append({
        'Travel Time': f'{time} min',
        'Transit Zero-Access': (transit_df['w_c000_19'] == 0).sum(),
        'Transit %': (transit_df['w_c000_19'] == 0).mean() * 100,
        'Auto Zero-Access': (auto_df['w_c000_19'] == 0).sum(),
        'Auto %': (auto_df['w_c000_19'] == 0).mean() * 100,
    })

zero_df = pd.DataFrame(zero_access)
print("⚠️ ZERO-ACCESS BLOCK ANALYSIS")
print("=" * 60)
zero_df

In [ ]:
# Identify the lowest-access blocks
transit_30 = load_accessibility_data('transit', 30)
low_access_blocks = transit_30[transit_30['w_c000_19'] < transit_30['w_c000_19'].quantile(0.10)][['id', 'w_c000_19']]
low_access_blocks = low_access_blocks.sort_values('w_c000_19')

print(f"📍 LOWEST 10% TRANSIT ACCESS BLOCKS ({len(low_access_blocks):,} blocks)")
print("=" * 50)
print(f"These blocks can access fewer than {transit_30['w_c000_19'].quantile(0.10):,.0f} jobs in 30 min by transit")
print(f"\nSample of lowest-access block IDs:")
print(low_access_blocks.head(10))

---
## 9. Summary & Key Findings

In [ ]:
# Generate summary statistics
print("="*70)
print("📊 MIAMI-DADE TRANSIT ACCESSIBILITY - KEY FINDINGS")
print("="*70)

auto_30 = load_accessibility_data('automobile', 30)
transit_30 = load_accessibility_data('transit', 30)
bike_lts1_30 = load_accessibility_data('bike_lts1', 30)
bike_lts4_30 = load_accessibility_data('bike_lts4', 30)

print("\n🚗 AUTO-TRANSIT GAP")
print("-"*40)
auto_mean = auto_30['w_c000_19'].mean()
transit_mean = transit_30['w_c000_19'].mean()
print(f"  Auto avg jobs (30 min):     {auto_mean:>12,.0f}")
print(f"  Transit avg jobs (30 min):  {transit_mean:>12,.0f}")
print(f"  Gap ratio:                  {auto_mean/transit_mean:>12.1f}x")

print("\n📈 TRANSIT EQUITY")
print("-"*40)
gini = compute_gini(transit_30['w_c000_19'].values)
print(f"  Gini coefficient:           {gini:>12.3f}")
print(f"  P90/P10 ratio:              {transit_30['w_c000_19'].quantile(0.90)/transit_30['w_c000_19'].quantile(0.10):>12.1f}x")
print(f"  Zero-access blocks:         {(transit_30['w_c000_19']==0).sum():>12,}")

print("\n🚲 BICYCLE INFRASTRUCTURE GAP")
print("-"*40)
lts1_mean = bike_lts1_30['w_c000_19'].mean()
lts4_mean = bike_lts4_30['w_c000_19'].mean()
print(f"  LTS1 (safe) avg jobs:       {lts1_mean:>12,.0f}")
print(f"  LTS4 (any route) avg jobs:  {lts4_mean:>12,.0f}")
print(f"  Safe infrastructure gap:    {lts4_mean/lts1_mean:>12.1f}x")

print("\n💰 WAGE ACCESSIBILITY (30 min transit)")
print("-"*40)
low_wage = transit_30['w_ce01_19'].mean()
high_wage = transit_30['w_ce03_19'].mean()
print(f"  Low-wage jobs accessible:   {low_wage:>12,.0f}")
print(f"  High-wage jobs accessible:  {high_wage:>12,.0f}")
print(f"  High/Low ratio:             {high_wage/low_wage:>12.2f}x")

print("\n" + "="*70)
print("✅ EDA COMPLETE")
print("="*70)

---
## 10. Next Steps for Capstone

### Recommended Analyses:
1. **Spatial Mapping**: Visualize zero-access and low-access blocks on a map
2. **Demographic Overlay**: Join with census data to identify equity populations
3. **Temporal Comparison**: Compare 2018 vs 2019 trends
4. **Scenario Modeling**: Simulate service improvements
5. **Composite Index**: Create an equity accessibility score

### Key Equity Questions to Explore:
- Which neighborhoods have the largest auto-transit gaps?
- Are low-income areas disproportionately transit-dependent?
- How does bicycle infrastructure quality correlate with demographics?
- What service changes would most improve equity?